In [ ]:
import os
import qca
import uuid
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
%load_ext line_profiler

In [ ]:
cmds = []
outfiles = []
ngpus = 8
gpus = list(range(ngpus))
ncuda_mps_procs_per_gpu = 1
ics = [1, 3, 5, 6, 7, 9, 11, 15]
rules = list(range(0, 33, 2))[1:]
nrows = 5
ncols = 5
nqubits = int(nrows * ncols)
ncycles = 50
open_boundaries = "true"
gate_mode = 5
gate_err = 0.0  # 2.0
trajectories = range(1)  # range(80)
rmajor = "true"
verbose = "false"
execute = False
count = 0
for trajectory in trajectories:
    for ic in ics:
        for rule in rules:
            ngpus = len(gpus)
            ncuda_mps_procs = ncuda_mps_procs_per_gpu * ngpus
            gpu_id = gpus[int(count % ngpus)]
            (cmd, stdout, stderr, outfile) = qca.run.circuit.cycle(
                ncycles=str(ncycles),
                nrows=str(nrows),
                ncols=str(ncols),
                rule=str(rule),
                ic=str(ic),
                gate_mode=str(gate_mode),
                open_boundaries=str(open_boundaries),
                trajectory=str(trajectory),
                gate_err=str(gate_err),
                rmajor=str(rmajor),
                verbose=str(verbose),
                execute=execute,
                qca_root="./"
            )
            # cmd = (f"CUDA_VISIBLE_DEVICES={str(gpu_id)} {cmd} &> " +
            #        f"${{WORKING_DIR}}/" +
            #        f"{str(gpu_id)}id_{str(nqubits)}qubits_{str(nrows)}r_{str(ncols)}c_{str(rule)}ru_{str(ic)}ic.out &")
            cmd = (f"CUDA_VISIBLE_DEVICES={str(gpu_id)} srun {cmd} &> " +
                   f"${{WORKING_DIR}}/" +
                   f"{str(gpu_id)}id_{str(nqubits)}qubits_{str(nrows)}r_{str(ncols)}c_{str(rule)}ru_{str(ic)}ic.out &")
            count = count + 1
            if count % ncuda_mps_procs == 0:
                cmd = (f"{cmd} \n" +
                       "wait \n" +
                       "date \n")
            cmds.append(cmd)
            outfiles.append(outfile)

preamble = ("#!/usr/bin/env bash \n" +
            f"#SBATCH --job-name={str(uuid.uuid4()).split('-')[0]} \n" +
            "#SBATCH --nodes=1 \n" +
            "#SBATCH --exclusive \n" +
            "#SBATCH --partition=bigjob \n" +
            "#SBATCH --time=10:00:00 \n" +
            "#SBATCH --export=ALL \n" +
            "#SBATCH --output=%j.out \n" +
            "#SBATCH --error=%j.err \n" +
            # f"export WORKING_DIR=$(pwd)/{str(uuid.uuid4()).split('-')[0]} \n" +
            "export WORKING_DIR=${SLURM_SUBMIT_DIR}/${SLURM_JOB_ID} \n" +
            "mkdir -p ${WORKING_DIR} \n\n" +
            "echo \"hostname: $(hostname)\" \n\n" +
            "date \n")
postamble = "date \n"
with open("qca.sh", "+w") as file:
    file.write(preamble)
    for cmd in cmds:
        file.write(f"{cmd} \n")
    file.write(postamble)

In [ ]:
single_qubit_files = list(it.chain.from_iterable([[outfile[0]
                                                   for outfile in outfileset]
                                                  for outfileset in outfiles]))
two_qubit_files = list(it.chain.from_iterable([[outfile[1]
                                                for outfile in outfileset]
                                               for outfileset in outfiles]))

In [ ]:
%%time
single_qubit_densities = qca.read.density.qubit_densities(single_qubit_files)

In [ ]:
%%time
two_qubit_densities = qca.read.density.qubit_densities(two_qubit_files)

In [ ]:
count = 0
reshaped_single_qubit_densities = []
reshaped_two_qubit_densities = []
for trajectory in trajectories:
    for ic in ics:
        for rule in rules:
            for cycle in range(ncycles):
                reshaped_single_qubit_densities.append((rule, ic, cycle, single_qubit_densities[count]))
                reshaped_two_qubit_densities.append((rule, ic, cycle, two_qubit_densities[count]))
                count = count + 1
reshaped_single_qubit_densities = np.array(reshaped_single_qubit_densities, dtype=object)
reshaped_two_qubit_densities = np.array(reshaped_two_qubit_densities, dtype=object)

In [ ]:
np.save("data/single_qubit_densities_0.00err", reshaped_single_qubit_densities)
np.save("data/two_qubit_densities_0.00err", reshaped_two_qubit_densities)

In [ ]:
single_qubit_densities = np.load("data/single_qubit_densities_0.00err.npy", allow_pickle=True)
two_qubit_densities = np.load("data/two_qubit_densities_0.00err.npy", allow_pickle=True)

In [ ]:
density_index = 3
single = [single_qubit_densities[index, density_index] for index in range(len(single_qubit_densities))]
two = [two_qubit_densities[index, density_index] for index in range(len(two_qubit_densities))]

In [ ]:
%%time
entropy_order = 2
mutual_information = qca.read.mutual.information(single, two, entropy_order)

In [ ]:
mutual_information = [(single_qubit_densities[index, :3], mutual_information[index])
                      for index in range(len(single_qubit_densities))]
mutual_information = np.array(mutual_information, dtype=object)

In [ ]:
np.save("data/mutual_information_0.00err", mutual_information)

In [ ]:
mutual_information = np.load("data/mutual_information_0.00err.npy", allow_pickle=True)

In [ ]:
tolerance = 0
clustering = qca.read.mutual.clustering([mutual_information[index, 1]
                                         for index in range(len(mutual_information))], tolerance)

In [ ]:
nsims = len(clustering) / ncycles
plt.figure(figsize=(16, 9));
plt.title("Clustering; all rules, all ics")
for sim in range(int(nsims)):
    sim_rule, sim_ic, _ = list(mutual_information[int(sim*ncycles), 0])  # simulation rule, simulation initial condition, cycle number
    sim_clustering = clustering[int(sim*ncycles):int((sim+1)*ncycles)]
    plt.xlabel("cycle");
    plt.xscale("linear");
    plt.xlim(0, 50);
    plt.ylabel("clustering");
    plt.yscale("log");
    plt.ylim(1e-9, 1);
    plt.scatter(range(ncycles), sim_clustering, label=f"rule: {sim_rule} ic: {sim_ic}");
# plt.legend();
# plt.savefig("2d_clustering_all_rules_all_ics.png", dpi=300)

In [ ]:
for sim_index in range(len(mutual_information)//ncycles):
    print(f"sim_index: {sim_index}")
    sim_rule, sim_ic, _ = list(mutual_information[int(sim_index*ncycles), 0])
    sim_clustering = clustering[int(sim_index*ncycles):int((sim_index+1)*ncycles)]
    plt.figure(figsize=(8, 4));
    plt.title(f"Clustering; {sim_rule}ru {sim_ic}ic");
    plt.xlabel("cycle");
    plt.ylabel("clustering");
    plt.scatter(range(ncycles), sim_clustering, color="black", label=f"rule: {sim_rule} ic: {sim_ic}");
    plt.legend();
    # plt.savefig(f"2d_clustering_{sim_rule}ru_{sim_ic}ic.png")
    plt.show();

In [ ]:
## TODO: add entropy and polarization methods to the QCA Python module
entropy_order = 2
def polarization(densities_list, index):
    polarization_list = []
    for densities in densities_list:
        pol = qca.read.mutual.polarization(densities, index)
        polarization_list.append(pol)
    return polarization_list
def _entropy(densities, alpha):
    return [qca.read.mutual._calculate_entropy(density[1], alpha)
            for density in densities]
def entropy(densities_list, alpha):
    entropy_list = []
    for densities in densities_list:
        ent = qca.read.mutual.entropy(densities, alpha)
        entropy_list.append(ent)
    return entropy_list

site_polarization = np.array(polarization(single, 1), dtype=object)[:, :, 1]
site_entropy = np.array(entropy(single, entropy_order), dtype=object)[:, :, 1]

In [ ]:
sim_index = 1  # 238, 239 are interesting
sim_rule, sim_ic, _ = list(mutual_information[int(sim_index*ncycles), 0])
sim_polarization = site_polarization[int(sim_index*ncycles):int((sim_index+1)*ncycles)]
for cycle in range(ncycles):
    plt.figure(figsize=(8, 4));
    plt.title(f" {sim_rule}ru {sim_ic}ic")
    plt.xlabel("qubit index");
    plt.xlim(-1, 25);
    plt.ylabel("prob. of |1>");
    plt.ylim(-0.05, 1.05);
    plt.scatter(range(nqubits), np.abs(sim_polarization[cycle]), color="black");
    # plt.savefig(f"gifs/spin/Site_spin_{sim_rule}ru_{sim_ic}ic_{cycle}cycle.png");
    plt.show()

In [ ]:
sim_index = 0  # 238, 239 are interesting
sim_rule, sim_ic, _ = list(mutual_information[int(sim_index*ncycles), 0])
sim_polarization = site_polarization[int(sim_index*ncycles):int((sim_index+1)*ncycles)]
for cycle in range(ncycles):
    plot_polarization = np.array(np.abs(sim_polarization[cycle].reshape([nrows, ncols], order="F")), dtype=float)
    plt.figure(figsize=(10, 10))
    plt.rc("font", family="serif")
    plt.rc("mathtext", fontset="cm")
    plt.title(f"Site-Local Polarization; rule {sim_rule}, ic {sim_ic}")
    plt.imshow(
        plot_polarization,
        cmap="hot",
        interpolation="none",
        extent=(0, nrows, 0, ncols),
        aspect="auto")
    cbar = plt.colorbar()
    plt.clim(0,1)
    cbar.ax.set_title(r"$ | \langle 1 | \psi \rangle |^{2} $", fontsize=16);
    # plt.savefig(f"gifs/spin/Site_spin_{sim_rule}ru_{sim_ic}ic_{cycle}cycle.png");
    plt.show()

In [ ]:
sim_index = 1  # 238, 239 are interesting
sim_rule, sim_ic, _ = list(mutual_information[int(sim_index*ncycles), 0])
sim_entropy = site_entropy[int(sim_index*ncycles):int((sim_index+1)*ncycles)]
for cycle in range(ncycles):
    plt.figure(figsize=(8, 4));
    plt.title(f"Site entropy {sim_rule}ru {sim_ic}ic")
    plt.xlabel("qubit index");
    plt.xlim(-1, 25);
    plt.ylabel("entropy");
    plt.ylim(-0.01, 1);
    plt.scatter(range(nqubits), np.abs(sim_entropy[cycle]), color="black");
    # plt.savefig(f"gifs/entropy/Site_entropy_{sim_rule}ru_{sim_ic}ic_{cycle}cycle.png");
    plt.show()